# Segunda entrega
Este proyecto consiste de un algoritmo genético
que busca aproximar una imagen dada usando triángulos.

## Descripción, estructura general

El algoritmo genera una primera generación aleatoriamente,
luego selecciona el mejor para pasar a la siguiente generación
y el resto de esta generación se obtiene a traves del cruce de dos
individuos.

En el cruce, los individuos tienen mayor posibilidad de reproducirse,
usando un método de ruleta para su elección, según mayor sea su fitness,
es decir, según más se parezcan a la imágen dada.

Al ocurrir un cruce existe la posibilidad de que los dos inviduos
creados por dicho cruce sean mutados. Las mutaciones transforman
uno o varios genes no mas del 5% y tienen una probabilidad del 5% de ocurrir.

Se cruzan tantos individuos como haga falta para recuperar la cantidad
original y una vez hecho esto se procede a generar otra generación
tomando a esta anterior (y no una aleatoria) como los 'padres'.

## Funciones

Las funciones que implementan lo descrito anteriormente se encuentran más
abajo.

Para estas funciones son centrales 3 bibliotecas:

- Numpy, para trabajar con ciertos arreglos y graficar.
- Computer Vision (cv2), para dibujar polígonos y otras cosas relacionadas
  con imágenes.
- skimage, también para la manipulación de imágenes.

Hasta ahora ninguna de las funciones posee manejo de errores.

## Resultados

Con la implementación actual no se logro aproximar ninguna imagen,
el algoritmo no logra converger incluso para las imágenes más sencillas
y pequeñas: intenté con una imagen de 80x80 píxeles que consiste
de un cuadrado en fondo blanco. 
El fitnnes del mejor individuo por generación (véase más abajo)
nunca supero 0.6 (debería haberse hacercado a 1) y, en consecuencia,
las imágenes producidas nunca llegaron a parecerse a la imágen dada.

En el proceso de probar el programa se intentaron muchas configuraciones.
Aumentar o dismunuir la población o la cantidad de triángulos de cada
invividuo.
Cambiar la forma en que el fitness de cada individuo es calculado.
Cambiar como eran elegidos los invidiuos para reproducirse.
Cambiar la probabilidad de mutación y que tan extensivas estas podían ser.
Y algunas otras cosas más.
Al final de cada una de las pruebas el programa seguía sin comportarse
adecuadamente.

Desconozco en este momento la causa de su mal funcionamiento.

## Colab y ejecución local

En algunas partes del proyecto hay partes de código
comentadas que use para ejecutar el proyecto localmente
que no quizas no funcionen en Google Colab.

Si se desea ejecutar este proyecto,
recomiendo exportarlo como un script de python (.py)
y descomentar las líneas apropiadas para probarlo en
local (las versiones anteriores al final deben comentarse,
si se exporta todo a un solo archivo).
Si se quiere probar en local, los ejemplos de abajo
utilizan una imágen que se puede obtener del (github)[]
del proyecto.

## Versiones anteriores

Al final se pueden encontrar versiones anteriores de las funciones del proyecto. Algunas versiones puede que no las haya guardado.

# Leer una imágen

Esta función toma un string que especifíca
el lugar donde se encuentra la imágen.
Puede ser un lugar en un sistema de archivos local
o un enlace de internet.


Devuelve:

1. La imagen como un array legible por `cv2`.
2. La dimensión X de la imagen
3. La dimensión Y de la imagen

In [ ]:
'''
Reads an image from specified location.

Args:
	image: location of image, either local or public URL.

Returns:
	imgread: numpy array containing image
	X: horizontal size of image
	Y: vertical size of image
'''
def read_img(image: str) -> [list,int,int]:
    from skimage import io
    import numpy as np
    import cv2

    img = io.imread(image)

    X = img.shape[0]
    Y = img.shape[1]

    imgread = [img,X,Y]

    return imgread

# Test
#print(read_img("./Freedo_improved.jpeg")[0])
#print(read_img("./Freedo_improved.jpeg")[1])
#print(read_img("./Freedo_improved.jpeg")[2])


# Dibujar lista de triángulos

La siguiente función dibuja una lista
de triángulos dada sobre un cuadrado blanco
y devuelve la imagén en un array de numpy.

La lista de triángulos debe tener cuatro elementos:
los tres vértices y el color. El color debe ser el último
elemento.

Los vértices son pares de enteros de la forma `(N,M)`.
Los colores son una tripleta RBG, es decir,
una tripleta de enteros cuyos valores están entre cero
y 255.

In [ ]:
'''
Draw a list of triangles in a white X*Y square.

Args:
	X: horizontal dimension of image
	Y: vertical dimension of image
	triangles: list of triangles. This list must be 4
		element list. The first three are the vertices x,y
		coordinates and the last is the color in RBG format.

Returns:
    image: the image with the triangles drawn, as an array.
'''
def draw_triangles(X,Y,triangles):
    import numpy as np
    import cv2

    image = np.ones((X,Y,3), np.uint8)*255 # Blank squared image

    N = len(triangles)

    for i in range(0,N):
        pts =  np.array(
                [triangles[i][0],triangles[i][1],triangles[i][2]],
                np.int32
               )
        pts = pts.reshape((-1,1,2))
        cv2.fillPoly(image, [pts], (triangles[i][3]))

    return image

# Primera generación

La siguiente función toma una cantidad de triángulos (`N`),
de población (`P`) y las dimensiones `X,Y` de las imagenes
que se quieren crear.

Con esto la función crea aleatoriamente `P` imágenes,
las cuales contienen `N` triángulos y son de tamaño
`X*Y`.

El programa devuelve una lista de imágenes como arreglos
de numpy junto con una lista de triángulos,
donde los elementos de estas dos listas se corresponden
uno a uno.

In [ ]:
'''
Create the first generation of individuals, randomly.

Args:
	N: amount of triangles per individual, an integer
	P: amount of individuals, an integer
	X: horizontal size of images
	Y: vertical size of images

Returns:
	imagearr: list of images in numpy array format
	triangles: list of the lists of triangles that compose the images
		in imagearr. the nth element is the list of triangles of the
		nth image in imagearr.
'''
def first_gen(N: int, P: int, X: int, Y: int) -> [list,list]:
    import random

    triangles = []
    imagearr = []
    for i in range(0,P):
        vertex = []
        for i in range(0,N):
            triag = [
                [random.randint(0,X),random.randint(0,Y)],
                [random.randint(0,X),random.randint(0,Y)],
                [random.randint(0,X),random.randint(0,Y)],
                [random.randint(0,255),random.randint(0,255),random.randint(0,255)]
            ]
            vertex += [triag]
        imagearr += [draw_triangles(X,Y,vertex)]
        triangles += [vertex]

    return [imagearr,triangles]

# Test
#first_gen(5,5,204,209)[0][0]

# Función para calcular el 'fitness'

La siguiente función toma dos imágenes,
dadas como arrays de numpy,
y devuelve un coeficiente real que cuantifica
lo parecidas que son las imágenes,
donde 1 es concidencia perfecta.

La función compara pixel a pixel las dos imágenes,
rellenando una lista `difflist` compuesta por los entero 0 y 1,
donde 1 representa que las imágenes coinciden en los
valores de dicho pixel y 0 que difieren.

Se calcula la suma total de elementos en `difflist`,
y esta suma dividida por la longitud de dicha lista
da un coeficiente real que expresa del 0 al 1
cuantos de los píxeles de `image` coinciden con
los de `original`.

## Versiones anteriores

Esta función cambió tres veces hasta llegar a su versión
actual. 

Al principio de usaba el método `cv2.subtract`
para 'restar' las imágenes. Nunca llegué a entender
como funciona dicha función de `cv2` y, dado que el
algoritmo no estaba convergiendo, decidí cambiarla.

La segunda versión paso a utilizar el método `structural_similarity`
de `skimage.metrics` que calcula el [índice de parecido estructural](https://en.wikipedia.org/wiki/Structural_similarity).

Nuevamente el método me pareció muy complicado y decidí
finalmente utilizar una simple comparación pixel a pixel.

In [ ]:
'''
Compares two images of same size pixel by pixel.

Args:
	original: original image we want to compare to,
		given as a numpy array.
	image: image we wish to compare given as a numpy array.

Returns:
	diff: real number indicating the percentage of pixels
		that coincide between the two images.
'''
def fitness(original, image):

    import numpy as np
    from skimage import io
    import cv2

    X = original.shape[0]
    Y = original.shape[1]

    difflist = []
    for i in range(0,X):
        for k in range(0,Y):
            origvals = []
            imgvals = []
            for l in range(0,3):
                origvals += [original.item(i,k,l)]
                imgvals += [image.item(i,k,l)]
            if origvals[0] == imgvals[0] and origvals[1] == imgvals[1] and origvals[2] == imgvals[2]:
                difflist += [1]
            else:
                difflist += [0]

    diffsum = sum(difflist)
    diff = sum(difflist)/len(difflist)

    return diff
# Test
#fitness(
#    read_img("./Freedo_improved.jpeg")[0],
#    read_img("./Freedo_improved_inverted.jpeg")[0],
#)

# Mutaciones

La siguiente función recibe una lista
de triángulos y dos valores X,Y (enteros) 
que indican el rango
permitido para cambiar el valor de los vertices
de dichos triángulos, usualmente estos valores X,Y
son las dimensiones de la imágen en que se encuentran
dichos triángulos.

La función calcula aleatoriamente cuantos triángulos
modificar y cuantos atributos de dichos tríangulos
va a modificar. Y los atributos seleccionados de los
triángulos seleccionados los modifica no más del 5%.

Finalmente, devuelve la lista de triángulos modificada.

In [ ]:
'''
Mutates the attributes of triangles in a
list of triangles, no more than 5%.
The triangles and the attributes to modify are
selected randomly.

Args:
	triangles: a list, each element of which
		is a triangle, i.e., a list with four
		elements: the coordinates of three vertices
		and a color in RGB format.
	X: Maximum value allowed for the x coordinate
		of the vertices
	Y: Maximum value allowed for the y coordinate
		of the vertices

Returns:
	triangles: a list of triangles
'''
def mutate(triangles,X,Y):
    import random

    N = len(triangles)

    M = random.randint(0,N-1)
    K = random.randint(0,3)

    colorpercent = int(255 * 0.05)
    Xvertpercent = int(X * 0.05)
    Yvertpercent = int(Y * 0.05)

    for i in range (0,M):
        l = random.randint(0,N-1)
        for i in range (0,K):
            k = random.randint(0,3)
            if k != 3:
                triangles[l][k][0] += random.randint(1,Xvertpercent)*random.choice([-1,1])
                triangles[l][k][1] += random.randint(1,Yvertpercent)*random.choice([-1,1])
                if triangles[l][k][0] >= X:
                    triangles[l][k][0] -= triangles[l][k][0] - X
                if triangles[l][k][1] >= Y:
                    triangles[l][k][1] -= triangles[l][k][1] - Y
            else:
                triangles[l][k][0] += random.randint(1,colorpercent)*random.choice([-1,1])
                triangles[l][k][1] += random.randint(1,colorpercent)*random.choice([-1,1])
                triangles[l][k][2] += random.randint(1,colorpercent)*random.choice([-1,1])
                if triangles[l][k][0] >= 255:
                    triangles[l][k][0] -= triangles[l][k][0] - 255
                if triangles[l][k][1] >= 255:
                    triangles[l][k][1] -= triangles[l][k][1] - 255
                if triangles[l][k][2] >= 255:
                    triangles[l][k][1] -= triangles[l][k][1] - 255

    return triangles

# Test
# triangles = first_gen(5,5,204,209)[1][2]
# print(triangles)
# mutate(triangles,204,209)

# Seleccionar individuos para reproducirse
Esta función se encarga de,
dada una lista de imágenes,
elegir dos dependiendo de su fitness
usando el método de la "ruleta":
individuos con mayor fitness tienen
más probabilidad de ser elegidos.

Este método de ruleta se implementó
sumando el fitness de tomas las imágenes
en la lista, llamemosle `diffsum` a esta
suma, y después construyendo un intervalo
de cero a `diffsum` particionado
por las sumas parciales de los fitness
de cada individuo, empezando por el mejor
y en orden descendente.
Luego se elige un número al azar entre cero
y `diffsum` y el extremo superior del intervalo
donde caiga dicho número será el fitness del
individuo escogido.

Además de esto, la función regresa
una variable `best` que indica el fitness
y la posición (en `imagearr`, la lista de imágenes)
del mejor individuo.

## Versiones anterioes
Esta función en su primera versión
se encargaba de refinar la lista de invididuos
que irían a formar parte de la siguiente generación.

Dependiendo de su fitness, elegía con mayor probabilidad
a los individuos con mayor fitness.
De esta forma se elegían una cantidad de individuos
menor que la población, y esta cantidad se completaba
haciendo cruces para llenar la siguiente generación.

Como el programa no pudo converger, una de las primeras
cosas que pensé en cambiar fue esta función.

In [ ]:
'''

'''
def selection(original,imagearr) -> [int]:
    import random

    N = len(imagearr)

    difflist = []
    for i in range(0,N):
        difflist += [[fitness(original,imagearr[i]),i]]

    difflist = sorted(difflist, reverse=True)
    best = difflist[0]

    selected = []

    diffsum = 0
    problist = []
    for i in range(0,N):
        diffsum += difflist[i][0]
        problist += [[diffsum,difflist[i][1]]]

    while len(selected) < 2:
        end = N-1
        start = 0
        r = random.uniform(0,diffsum)

        while end != start+1:
            mid = (end+start)//2
            if r > problist[mid][0]:
                start = mid
            elif r < problist[mid][0]:
                end = mid
            else:
                end = start+1
        selected += [problist[end][1]]

    return [selected,best]

# Test
#selection(read_img("./Freedo_improved.jpeg")[0],first_gen(12,10,204,209)[0])

In [ ]:
def crossover(parentA:list, parentB:list, X: int, Y: int,):
    import random

    N = len(parentA)

    sonA = []
    sonB = []

    for i in range(0,N):
        if i <= N//2:
            sonA += [parentA[i]]
            sonB += [parentB[i]]
        else:
            sonB += [parentA[i]]
            sonA += [parentB[i]]

    if random.uniform(0,100) <= 7:
        if random.randint(0,1) == 1:
            sonA = mutate(sonA,X,Y)
        else:
            sonB = mutate(sonB,X,Y)

    return [sonA,sonB]

# Test
#crossover(first_gen(15,7,204,209)[1][0],first_gen(15,7,204,209)[1][1],204,209)

In [ ]:
def next_gen(original,imagearr,triangles):
    import cv2

    img = original
    X = img.shape[0]
    Y = img.shape[1]

    N = len(imagearr)

    nextgentriag = []
    nextgentriag += [triangles[selection(original,imagearr)[1][1]]]

    while len(nextgentriag) < N:
        k = selection(original,imagearr)[0]
        sons = crossover(triangles[k[0]],triangles[k[1]],X,Y)
        nextgentriag += [sons[0]]
        nextgentriag += [sons[1]]

    while N != len(nextgentriag):
        nextgentriag = nextgentriag[:-1]

    nextgenimgarr = []

    for i in range(0,len(nextgentriag)):
        nextgenimgarr += [draw_triangles(X,Y,nextgentriag[i])]

    best = selection(original,nextgenimgarr)[1]

    return [nextgenimgarr,nextgentriag,best]
    
# Test
next_gen(
    read_img("./simple-rectangle.png")[0],
    first_gen(4,5,80,80)[0],
    triangles)[1]

In [ ]:
def gen_algo(original: str, N: int, P: int):
    from matplotlib import pyplot as plt
    import cv2

    X = read_img(original)[1]
    Y = read_img(original)[2]
    img = read_img(original)[0]

    firstgen = first_gen(N,P,X,Y)

    nextgen = next_gen(img,firstgen[0],firstgen[1])

    difflist = []
    bestlist = []
    L = 500
    for i in range(0,L):
        nextgen = next_gen(img,nextgen[0],nextgen[1])
        difflist += [nextgen[2][0]]
        if i % 5 == 0:
            print(i)
        if i % 10 == 0:
            cv2.imwrite("./best-images/simple-square"+str(i)+".png",nextgen[0][nextgen[2][1]])

    plt.plot(difflist)
    plt.show()

    #cv2.imshow("window2",nextgen[0][0])
    #cv2.waitKey(0)
    #cv2.destroyWindow('window2')
    #cv2.waitKey(1)

    #return nextgen

# Test
gen_algo("./simple-rectangle.png",15,15)

# Licencia y copyright

Todo el software de este proyecto esta bajo al GPL de GNU

Copyright (C) Jhonny Lanzuisi (jabl97@gmail.com) 2021

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published 
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

# Versiones anteriores

In [ ]:
def old_read_img(image: str) -> [list,int,int,float]:
    from skimage import io
    import numpy as np
    import cv2

    img = io.imread(image)

    X = img.shape[0]
    Y = img.shape[1]
    pixsum = np.sum(img)

    imgread = [img,X,Y,pixsum]
    
    return imgread

# Test
print(read_img("./Freedo_improved.jpeg")[0])
print(read_img("./Freedo_improved.jpeg")[1])
print(read_img("./Freedo_improved.jpeg")[2])
print(read_img("./Freedo_improved.jpeg")[3])

In [ ]:
def old_fitness(original, image):
    from skimage.metrics import structural_similarity as ssim
    import numpy as np
    import cv2
    (score,diff) = ssim(original, image, full=True, multichannel=True)
    
    return abs(score)

In [ ]:
def old_selection(original,imagearr) -> [int]:
    import random
    
    N = len(imagearr)

    difflist = []
    for i in range(0,N):
        difflist += [[fitness(original,imagearr[i]),i]]

    difflist = sorted(difflist, reverse=True)

    selected = []
    selected += [difflist[0][0]]

    diffsum = 0
    problist = []
    for i in range(1,N):
        diffsum += difflist[i][0]
        problist += [[diffsum,difflist[i][1]]]

    
    M = int(N * 0.9)
    for i in range(0,M):
        end = N-1
        start = 0
        r = random.uniform(0,diffsum)
        while end != start+1:
            mid = (end+start)//2
            if r > problist[mid][0]:
                start = mid
            elif r < problist[mid][0]:
                end = mid
            else:
                end = start+1
        selected += [problist[end][1]]
    
    # https://www.w3schools.com/python/python_howto_remove_duplicates.asp
    selected = list(dict.fromkeys(selected))

    return selected

# Test
selection(read_img("./Freedo_improved.jpeg")[0],first_gen(12,10,204,209)[0])

In [ ]:
def old_next_gen(original,imagearr,triangles):
    import cv2

    img = original
    X = img.shape[0]
    Y = img.shape[1]
    
    selected = selection(original,imagearr)
    
    N = len(imagearr)

    nextgentriag = []
    for i in range(0,len(selected)):
        nextgentriag += [triangles[selected[i]]]

    i = 1
    while len(nextgentriag) < N:
        sons = crossover(nextgentriag[i],nextgentriag[i+1],X,Y)
        nextgentriag += [sons[0]]
        nextgentriag += [sons[1]]
        i += 1

    while N != len(nextgentriag):
        nextgentriag = nextgentriag[:-1]

    nextgenimgarr = []
    
    for i in range(0,N):
        nextgenimgarr += [draw_triangles(X,Y,nextgentriag[i])]

    return [nextgenimgarr,nextgentriag]
    
# Test
#next_gen(read_img("./Freedo_improved.jpeg")[0],first_gen(6,10,204,209)[0],first_gen(6,10,204,209)[1])[1]

In [ ]:
def old_gen_algo(original: str, N: int, P: int,):
    from matplotlib import pyplot as plt
    import cv2
    
    X = read_img(original)[1]
    Y = read_img(original)[2]
    img = read_img(original)[0]

    firstgen = first_gen(N,P,X,Y)

    nextgen = next_gen(img,firstgen[0],firstgen[1])

    difflist = []
    L = 10
    for i in range(0,L):
        nextgen = next_gen(img,nextgen[0],nextgen[1])
        difflist += [fitness(img,nextgen[0][0])]
        
    #print(difflist)   
        
    #for i in range(0,P):
        #difflist += [fitness(img,nextgen[0][i])]

    plt.plot(difflist)
    plt.show()

    cv2.imshow("window2",nextgen[0][0])
    cv2.waitKey(0)
    cv2.destroyWindow('window2')
    cv2.waitKey(1)

    #return nextgen

# Test
gen_algo("./Freedo_improved.jpeg",9,10)